### Requirements
-----

In [1]:
import pandas as pd
pd.__version__

'1.5.3'

In [2]:
import sqlite3
sqlite3.sqlite_version

'3.45.1'

## SQL-Like Window functions in Pandas

use **.groupby** and **.transform** to achieve SQL-like window functionality in Pandas

> A window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities.

### Window functions in SQL

rqmts:
- an aggregation function or calculation to apply to the target column (e.g. `SUM()`, `RANK()`) ;
- the `OVER()` keyword to initiate the partition function ;
- the `PARTITION BY` keyword which defines which data partition(s) to apply the aggregation function ;
- (optional) the `ORDER BY` keyword to define the required sorting within each data partition.

### Window functions in Pandas

rqmts:
- `.groupby` in Pandas is analogous to the `PARTITION BY` keyword in SQL ;
- `.transform` allows you to apply complex transformations.

`.transform` takes a function as an argument. The function supplied to `.transform` can either be a string (for simple aggregation functions like sum, mean, count) or a lambda function for more complex operations.

### Examples

In [3]:
import re
df = pd.read_csv("data/web.csv",sep="|",header=0)
cols_to_drop = [col for col in df.columns if re.search('_visit', col)]
df.drop(cols_to_drop, axis=1, inplace=True)
df['day']=pd.to_datetime(df['day'])
print(df.shape)
df.sort_values(by=['identifier','day'],inplace=True)
df.reset_index(inplace=True)
df.head()

(3158, 7)


,index,hits,visits,day,identifier,orders,amount,product_pages
0,2155,1204458,160622,2019-04-16,3,54642,2.694153e+06,432140
1,2122,1211634,163821,2019-04-17,3,36422,1.742146e+06,437709
2,1985,1125891,155776,2019-04-18,3,49277,2.392967e+06,406391
3,2106,1115407,155006,2019-04-19,3,52085,2.589991e+06,404492
4,2148,1185745,168016,2019-04-20,3,79789,4.603390e+06,435203


**Save to SQLite Database**

In [4]:
with sqlite3.connect(":memory:") as conn:
    df.to_sql(name="web", con=conn, index=False)

**Example 1**:  Calculating the maximum number of transactions for each identifier in the dataset

In [5]:
ex1_sql_query = """
SELECT
    day
    , identifier
    , orders
    , MAX(orders) OVER(PARTITION BY identifier) as max_orders
FROM
    web
"""

In [6]:
# use pandas read_sql to execute the query and return a dataframe
ex1_sql = pd.read_sql(ex1_sql_query, con=conn)
print(ex1_sql.shape)
ex1_sql.head()

(3158, 4)


,day,identifier,orders,max_orders
0,2019-04-16 00:00:00,3,54642,195123
1,2019-04-17 00:00:00,3,36422,195123
2,2019-04-18 00:00:00,3,49277,195123
3,2019-04-19 00:00:00,3,52085,195123
4,2019-04-20 00:00:00,3,79789,195123


In [7]:
ex1_pandas = df.copy()

ex1_pandas["max_orders"] = ex1_pandas.groupby("identifier")["orders"].transform("max")

print(ex1_pandas.shape)
ex1_pandas.head()

(3158, 9)


,index,hits,visits,day,identifier,orders,amount,product_pages,max_orders
0,2155,1204458,160622,2019-04-16,3,54642,2.694153e+06,432140,195123
1,2122,1211634,163821,2019-04-17,3,36422,1.742146e+06,437709,195123
2,1985,1125891,155776,2019-04-18,3,49277,2.392967e+06,406391,195123
3,2106,1115407,155006,2019-04-19,3,52085,2.589991e+06,404492,195123
4,2148,1185745,168016,2019-04-20,3,79789,4.603390e+06,435203,195123


In [8]:
assert all(ex1_sql.max_orders == ex1_pandas.max_orders)

**Example 2**:  7 day - number of hits - moving average for each identifier

In [9]:
ex2_sql_query = """
SELECT
    day
    , identifier
    , hits
    , AVG(hits) OVER(
        PARTITION BY identifier
        ORDER BY day
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    )
     AS ma_7_day
FROM
    web
"""

In [10]:
ex2_sql = pd.read_sql(ex2_sql_query, con=conn)
print(ex2_sql.shape)
ex2_sql.head()

(3158, 4)


,day,identifier,hits,ma_7_day
0,2019-04-16 00:00:00,3,1204458,1204458.0
1,2019-04-17 00:00:00,3,1211634,1208046.0
2,2019-04-18 00:00:00,3,1125891,1180661.0
3,2019-04-19 00:00:00,3,1115407,1164347.5
4,2019-04-20 00:00:00,3,1185745,1168627.0


In [11]:
ex2_pandas = df.copy()

ex2_pandas["ma_7_day"] = (
    ex2_pandas.sort_values("day")
    .groupby("identifier")["hits"]
    .transform(lambda x: x.rolling(7, min_periods=1).mean())
)

print(ex2_pandas.shape)
ex2_pandas.head()

(3158, 9)


,index,hits,visits,day,identifier,orders,amount,product_pages,ma_7_day
0,2155,1204458,160622,2019-04-16,3,54642,2.694153e+06,432140,1204458.0
1,2122,1211634,163821,2019-04-17,3,36422,1.742146e+06,437709,1208046.0
2,1985,1125891,155776,2019-04-18,3,49277,2.392967e+06,406391,1180661.0
3,2106,1115407,155006,2019-04-19,3,52085,2.589991e+06,404492,1164347.5
4,2148,1185745,168016,2019-04-20,3,79789,4.603390e+06,435203,1168627.0


In [12]:
assert all(ex2_sql.ma_7_day == ex2_pandas.ma_7_day)

**Example 3**: Get previous day's number of visits for each identifier

In [13]:
ex3_sql_query = """
SELECT
    day
    , identifier
    , visits
    , LAG(visits, 1) OVER(
        PARTITION BY identifier
        ORDER BY day
    ) AS previous_visits
FROM
    web
"""

In [14]:
ex3_sql = pd.read_sql(ex3_sql_query, con=conn)
print(ex3_sql.shape)
ex3_sql.head()

(3158, 4)


,day,identifier,visits,previous_visits
0,2019-04-16 00:00:00,3,160622,NaN
1,2019-04-17 00:00:00,3,163821,160622.0
2,2019-04-18 00:00:00,3,155776,163821.0
3,2019-04-19 00:00:00,3,155006,155776.0
4,2019-04-20 00:00:00,3,168016,155006.0


In this example, we don't need to use the `transform` function because `shift` naturally returns a value for each row in the dataframe.

In [15]:
ex3_pandas = df.copy()

ex3_pandas["previous_visits"] = (
    ex3_pandas.sort_values("day").groupby("identifier")["visits"].shift(1)
)

print(ex3_pandas.shape)
ex3_pandas.head()

(3158, 9)


,index,hits,visits,day,identifier,orders,amount,product_pages,previous_visits
0,2155,1204458,160622,2019-04-16,3,54642,2.694153e+06,432140,NaN
1,2122,1211634,163821,2019-04-17,3,36422,1.742146e+06,437709,160622.0
2,1985,1125891,155776,2019-04-18,3,49277,2.392967e+06,406391,163821.0
3,2106,1115407,155006,2019-04-19,3,52085,2.589991e+06,404492,155776.0
4,2148,1185745,168016,2019-04-20,3,79789,4.603390e+06,435203,155006.0


In [16]:
assert all(ex3_sql.previous_visits.fillna(0).values == ex3_pandas.previous_visits.fillna(0).values)

**Example 4**: Daily percentage increase / decrease in sales amount

In [17]:
ex4_sql_query = """
SELECT
    day
    , identifier
    , amount
    , ROUND( (amount*1.0/previous_amount - 1)*100, 2) AS daily_percentage
FROM
    (
    SELECT
        `index`
        , day
        , identifier
        , amount
        , LAG(amount,1) OVER(
            PARTITION BY identifier ORDER BY day
        ) AS previous_amount
    FROM
        web
)
"""

In [18]:
ex4_sql = pd.read_sql(ex4_sql_query, con=conn)
print(ex4_sql.shape)
ex4_sql.head()

(3158, 4)


,day,identifier,amount,daily_percentage
0,2019-04-16 00:00:00,3,2.694153e+06,NaN
1,2019-04-17 00:00:00,3,1.742146e+06,-35.34
2,2019-04-18 00:00:00,3,2.392967e+06,37.36
3,2019-04-19 00:00:00,3,2.589991e+06,8.23
4,2019-04-20 00:00:00,3,4.603390e+06,77.74


In [19]:
ex4_pandas = df.copy()

ex4_pandas["daily_percentage"] = (
    ex4_pandas.sort_values("day")
    .groupby("identifier")["amount"]
    .transform(lambda x: round( (x / x.shift(1) - 1) * 100, 2))
)

print(ex4_pandas.shape)
ex4_pandas.head()

(3158, 9)


,index,hits,visits,day,identifier,orders,amount,product_pages,daily_percentage
0,2155,1204458,160622,2019-04-16,3,54642,2.694153e+06,432140,NaN
1,2122,1211634,163821,2019-04-17,3,36422,1.742146e+06,437709,-35.34
2,1985,1125891,155776,2019-04-18,3,49277,2.392967e+06,406391,37.36
3,2106,1115407,155006,2019-04-19,3,52085,2.589991e+06,404492,8.23
4,2148,1185745,168016,2019-04-20,3,79789,4.603390e+06,435203,77.74


In [20]:
assert all(ex4_sql.daily_percentage.fillna(0).values == ex4_pandas.daily_percentage.fillna(0).values)